In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Yahoo S5

In [5]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [6]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
        x = layers.LSTM(32)(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
        x = layers.LSTM(32, return_sequences=True)(x)
        x = layers.LSTM(64)(x)
        x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
#         (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

yahoo_A1 0.5714285224489823 0.08333334134920505 0.1666666472222242
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.18181816198347142 0.049999994500000554 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.46153841893491365 0.14999999350000026 0.49999997619047704
yahoo_A1 0.6153845633136124 0.2839285605050551 0.21999999070000026
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.6666666133333363 0.35079363651969214 0.3666666479444452
yahoo_A1 0.33333330000000166 0.09999999400000033 -0.0
yahoo_A1 0.8749999398437528 0.3194444407903436 -0.0
yahoo_A1 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A1 0.399999960000002 0.11904761327947858 0.437499969531252
yahoo_A1 0.18181816198347142 0.049999994500000554 -0.0
yahoo_A1 0.3636363272727291 0.0625000016493049 -0.0
yahoo_A1 0.2222221975308647 0.06249999296875071 0.3888888456790167
yahoo_A1 0.7272726677685981 0.4145833100962317 0.5299999765500008
yahoo_A1 0.6153845633136124 0.08333334384093748 -0.0
yahoo_A1 0.71428565816326

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A3 0.4999999562500026 0.05555556362513912 0.0
yahoo_A3 0.5333332871111137 0.10416666935658608 -0.0
yahoo_A3 0.7058822989619408 0.245833327158099 0.4749999586250034
yahoo_A3 0.4999999562500026 0.05555556362513912 -0.0
yahoo_A3 0.8571428000000028 0.1250000216603509 -0.0
yahoo_A3 0.4285713887755125 0.05555556109988678 -0.0
yahoo_A3 0.736842050969532 0.2750992006086117 0.25833331809722293
yahoo_A3 0.6249999492187529 0.07142858313976339 -0.0
yahoo_A3 0.7142856581632683 0.1000000141904741 -0.0
yahoo_A3 0.0 0.0 -0.0
yahoo_A3 0.9999999000000052 0.8333332582677795 0.9999998500000176
yahoo_A3 0.9473683612188393 0.6020833100161656 -0.0
yahoo_A3 0.7777777216049413 0.2722222180723905 -0.0
yahoo_A3 0.16666664861111138 0.045454540495868265 -0.0
yahoo_A3 0.7058822989619408 0.34444441966669936 0.33333331555555634
yahoo_A3 0.15384613727810673 0.04166666215277823 -0.0
yahoo_A3 0.9090908512396723 0.16666669091558128 -0.0
yahoo_A3 0.4999999562500026 0.05555556362513912 -0.0
yahoo_A3 0.499999956250002

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A4 0.6666666148148177 0.11904762532024238 0.02777777638888895
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.4285713887755125 0.05555556109988678 -0.0
yahoo_A4 0.5714285224489823 0.10714286085033937 -0.0
yahoo_A4 0.4285713887755125 0.05555556109988678 -0.0
yahoo_A4 0.8888888296296326 0.3125000001273142 0.12499999062500056
yahoo_A4 0.7058822989619408 0.20626983973109564 0.03999999840000004
yahoo_A4 0.4999999562500026 0.05555556362513912 -0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.5882352456747433 0.06250001071706508 -0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.8888888296296326 0.4166666471759278 0.4999999375000063
yahoo_A4 0.7058822989619408 0.17460317560887328 -0.0
yahoo_A4 0.899999941500003 0.3194444433228112 -0.0
yahoo_A4 0.6249999492187529 0.1250000042494581 0.4999999416666726
yahoo_A4 0.736842050969532 0.2763888807561932 0.3519047400473934
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.3999999626666689 0.05000000514393849 -0.0
yahoo_A4 0.15384613727810673 0.04166666215277823 -0.0
yahoo_A4 0.4285713887755125 0.062500

In [7]:
yahoo_results = pd.DataFrame(total_scores)

In [8]:
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.372141,0.118406,0.115219
yahoo_A2,0.000000,0.000000,0.000000
yahoo_A3,0.619585,0.158426,0.072220
yahoo_A4,0.527761,0.113948,0.052521


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
        x = layers.LSTM(32)(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
        x = layers.LSTM(32, return_sequences=True)(x)
        x = layers.LSTM(64)(x)
        x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
#         (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.2535044200740443 0.1447806601986309 0.496892986858861
D2 0.27027024683202794 0.1430758257118818 0.5224966503580902


In [11]:
nasa_results = pd.DataFrame(total_scores)

In [12]:
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.253504,0.144781,0.496893
D2,0.270270,0.143076,0.522497


### SMD

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [14]:
for loader in [load_smd]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
        x = layers.LSTM(32)(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
        x = layers.LSTM(32, return_sequences=True)(x)
        x = layers.LSTM(64)(x)
        x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
#         (x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.21943571700356884 0.06161971811210573 0.49999999837062536
smd 0.1300812885451792 0.008766601878491102 0.10738636288987174
smd 0.1599999851520013 0.06945027615120641 0.45430939502032086
smd 0.15873013963775034 0.054326076248608496 0.5134792598687884
smd 0.06611569603169239 0.01709401687303675 0.5065502117875327
smd 0.4429529853520138 0.2074377908237624 0.4577452786027556
smd 0.2647058591911784 0.11059659097718144 0.44507614186600203
smd 0.1930501754595206 0.04655828879055052 0.23773304459930272
smd 0.18532816836958452 0.058510958597963084 0.3252422075259167
smd 0.28363633908892766 0.08436972606604197 0.49733176991578426
smd 0.1044176607022476 0.027524555330189233 0.49420776847819725
smd 0.2710622474312033 0.08909967542668079 0.2933352702495944
smd 0.22556388959240378 0.07662499893437098 0.37106765630034616
smd 0.08080807299935448 0.020835594001211342 0.40636363179541324
smd 0.18972330283554026 0.056273405926430854 0.4097415000233758
smd 0.02499999751041683 0.006329113710409663 0.4

In [15]:
smd_results = pd.DataFrame(total_scores)

In [16]:
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.162948,0.046043,0.382637
